# ABIDES-ROHAN Usage Demo

This is a test for basic use of abides_markets simulation. It uses the original code from [abides-jpmc-public](https://github.com/jpmorganchase/abides-jpmc-public).

In [ ]:
import numpy as np
import pandas as pd
from abides_core.utils import fmt_ts, ns_date, str_to_ns
from matplotlib import pyplot as plt

from rohan.config import SimulationSettings
from rohan.simulation import SimulationContext, SimulationOutput, SimulationResult, SimulationService
from rohan.simulation.abides_impl import AbidesConfigMapper

## Build runnable configuration

Here we are generating a default config, based on ABIDES RMSC-4. 
The config object is a dictionary containing key elements like, start time, end time, agents to be used, latency and computation delay models.

In [ ]:
config: SimulationSettings = SimulationSettings()
config.seed = 42
config.date = "20260130"
config.model_dump_json()

We can see the ABIDES config by explicitly building it, using AbidesConfigMapper:

In [ ]:
abides_wrapper = AbidesConfigMapper(config)
abides_config = abides_wrapper.build_configuration()

## Running simulation

Once the config is ready it can be run using the abides runner function.
It instanciates a simulation kernel, runs the configuration and returns an end_state that mostly contains pointers to the different agent objects.
The agents are in their final state, their internal variables can be accessed to extract informations of interests like logs.

In [ ]:
simulation_service: SimulationService = SimulationService()
sim_results: SimulationResult = simulation_service.run_simulation(settings=config)

## Retrieving results

First, explore if the simulation was succesful and retrieve relevant metadata:

In [ ]:
print("Simulation completed.")
print(f"Duration (s): {sim_results.duration_seconds}")
context: SimulationContext = sim_results.context
print(f"Simulation context: {context}")

if sim_results.error:
    print(f"Simulation error: {sim_results.error}")

if sim_results.result:
    results: SimulationOutput = sim_results.result
else:
    raise RuntimeError("Simulation failed; no results to display.")

### Order book history L1

L1 data snapshots for every tick can be extracted
( best bid and ask price and quantity )

In [ ]:
l1: pd.DataFrame = results.get_order_book_l1()
l1.set_index("timestamp", inplace=True)
l1.head()

Here we plot the time series of the best bid and best ask price thoughout the simulation

In [ ]:
plt.plot(l1.time, l1.bid_price, label="Bid Price")
plt.plot(l1.time, l1.ask_price, label="Ask Price")
plt.legend()
plt.title("Level 1 Order Book Prices Over Time")
plt.xlabel("Time")
plt.ylabel("Price")

time_mesh = np.arange(str_to_ns(config.start_time), str_to_ns(config.end_time), 1e9 * 60 * 10)
_ = plt.xticks(time_mesh, [fmt_ts(time).split(" ")[1] for time in time_mesh], rotation=60)

L2 data snapshots for every tick can be extracted
( bids and asks price and quantity for every orderbook level. Here max depth logged is a parameter of the simulation and max number of levels we want to retrieve from the orderbook after the simulation is a parameter too)

In [ ]:
l2: pd.DataFrame = results.get_order_book_l2(n_levels=10)

As an illustration we plot the time series of the fifth best bid price and fifth best ask price throughout the simulation

In [ ]:
ask: pd.DataFrame = l2[(l2["level"] == 5) & (l2["side"] == "ask") & (l2["price"] > 0)]
ask.head()

In [ ]:
## plotting fifth best bid and fifth best ask
level = 5

plt.figure(figsize=(12, 6))
ask: pd.DataFrame = l2[(l2["level"] == level) & (l2["side"] == "ask") & (l2["price"] > 0)]
plt.scatter(
    ask["timestamp"],
    ask["price"],
    s=0.5,
)
bid: pd.DataFrame = l2[(l2["level"] == level) & (l2["side"] == "bid") & (l2["price"] > 0)]
plt.scatter(
    bid["timestamp"],
    bid["price"],
    s=0.5,
)
plt.ylim(bid["price"].mean() - 2 * bid["price"].std(), ask["price"].mean() + 2 * ask["price"].std())
plt.title(f"Level {level} Bid and Ask Prices Over Time")
plt.xlabel("Time")
plt.ylabel("Price")
plt.show()

# _ = plt.xticks(time_mesh, [fmt_ts(time).split(" ")[1] for time in time_mesh], rotation=60)

### Looking at agents logs

All agents can be inspected to retrieve desired information. 
The utility parse_logs_df for instance provides a quick way to retrieve and aggregate the log variables of each agent in a single dataframe

In [ ]:
logs_df = results.get_logs_df()
logs_df.sample(10)

#### Histogram of order submission times for noise agents

As an illustration we retrieve the submission times of all the orders sent by noise agent and display the histogram of all these times

In [ ]:
plt.figure(figsize=(12, 6))

# Extract ORDER_SUBMITTED events for NoiseAgent and convert time_placed to ns from midnight
noise_agent_orders = logs_df[(logs_df.agent_type == "NoiseAgent") & (logs_df.EventType == "ORDER_SUBMITTED")]
noise_agent_times = noise_agent_orders.time_placed.apply(lambda x: pd.Timestamp(x).value - ns_date(pd.Timestamp(x).value))

plt.hist(noise_agent_times, bins=50, alpha=0.7, edgecolor="black")

plt.xlabel("Time (ns from midnight)", fontsize=12)
plt.ylabel("Number of Orders", fontsize=12)
plt.title("NoiseAgent Order Submission Times", fontsize=14, fontweight="bold")

# Set x-axis to match the simulation time window
plt.xlim(str_to_ns("09:30:00"), str_to_ns("10:10:00"))
plt.xticks(time_mesh, [fmt_ts(time).split(" ")[1] for time in time_mesh], rotation=60)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Show summary statistics
print(f"Total NoiseAgent orders: {len(noise_agent_times)}")
if len(noise_agent_times) > 0:
    print(f"First order: {noise_agent_orders.time_placed.min()}")
    print(f"Last order: {noise_agent_orders.time_placed.max()}")

We proceed the same way for value agents as well

In [ ]:
plt.figure(figsize=(12, 6))

# Extract ORDER_SUBMITTED events for ValueAgent and convert time_placed to ns from midnight
value_agent_orders = logs_df[(logs_df.agent_type == "ValueAgent") & (logs_df.EventType == "ORDER_SUBMITTED")]
value_agent_times = value_agent_orders.time_placed.apply(lambda x: pd.Timestamp(x).value - ns_date(pd.Timestamp(x).value))

plt.hist(value_agent_times, bins=50, alpha=0.7, color="orange", edgecolor="black")

plt.xlabel("Time (ns from midnight)", fontsize=12)
plt.ylabel("Number of Orders", fontsize=12)
plt.title("ValueAgent Order Submission Times", fontsize=14, fontweight="bold")

# Set x-axis to match the simulation time window
plt.xlim(str_to_ns("09:30:00"), str_to_ns("10:10:00"))
plt.xticks(time_mesh, [fmt_ts(time).split(" ")[1] for time in time_mesh], rotation=60)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Show summary statistics
print(f"Total ValueAgent orders: {len(value_agent_times)}")
if len(value_agent_times) > 0:
    print(f"First order: {value_agent_orders.time_placed.min()}")
    print(f"Last order: {value_agent_orders.time_placed.max()}")